In [1]:
import pandas as pd

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = stopwords.words('english')


[nltk_data] Downloading package stopwords to C:\Users\Zoran
[nltk_data]     Barac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
patient_notes = pd.read_csv("patient_notes.csv")
features = pd.read_csv("features.csv")
train = pd.read_csv("train.csv")

In [4]:
train

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']
...,...,...,...,...,...,...
14295,95333_912,9,95333,912,[],[]
14296,95333_913,9,95333,913,[],[]
14297,95333_914,9,95333,914,['photobia'],['274 282']
14298,95333_915,9,95333,915,['no sick contacts'],['421 437']


In [5]:
train = train[train['annotation'] != '[]']
train


,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']
...,...,...,...,...,...,...
14291,95333_908,9,95333,908,['Nausea'],['354 360']
14293,95333_910,9,95333,910,['lives with roomate'],['576 594']
14297,95333_914,9,95333,914,['photobia'],['274 282']
14298,95333_915,9,95333,915,['no sick contacts'],['421 437']


In [6]:
labels = train.groupby(['case_num', 'pn_num'])['feature_num'].agg(list).reset_index()

# Rename the aggregated column to 'label'
labels.rename(columns={'annotation': 'label'}, inplace=True)

labels

,case_num,pn_num,feature_num
0,0,16,"[0, 1, 2, 3, 4, 6, 9, 10, 11, 12]"
1,0,41,"[1, 2, 3, 7, 9, 10, 11, 12]"
2,0,46,"[0, 1, 3, 4, 5, 6, 7, 9, 10, 11, 12]"
3,0,82,"[0, 1, 3, 4, 6, 7, 9, 10, 11, 12]"
4,0,100,"[0, 1, 2, 3, 4, 6, 9, 10, 11, 12]"
...,...,...,...
995,9,95145,"[901, 902, 904, 905, 906, 908, 912, 913, 914, ..."
996,9,95228,"[901, 902, 909, 910, 913, 914, 915]"
997,9,95243,"[900, 901, 902, 906, 908, 912, 913, 914, 916]"
998,9,95330,"[900, 901, 902, 904, 906, 907, 908, 912, 913, ..."


In [7]:
patient_notes

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...
...,...,...,...
42141,95330,9,Ms. Madden is a 20 yo female presenting w/ the...
42142,95331,9,A 20 YO F CAME COMPLAIN A DULL 8/10 HEADACHE T...
42143,95332,9,Ms. Madden is a 20yo female who presents with ...
42144,95333,9,Stephanie madden is a 20 year old woman compla...


In [9]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags

    text=re.sub("</?.*?>"," <> ",text)
    
    
    # remove special characters and digits
    text = re.sub(r'[\n]', '. ', text)

    text=re.sub("(\\|\\W)+"," ",text)

    text = re.sub(r'[\r]', '', text)

    text = text.replace('w/', '')
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('!', '')
    text = text.encode("ascii", "ignore").decode()
    text =  " ".join([word for word in str(text).split() if word not in stopwords])

    
    return text

patient_notes['text'] = patient_notes['pn_history'].apply(lambda x:pre_process(x))

#show the first 'text'
patient_notes['text']

0        17-year-old male, come student health clinic c...
1        17 yo male recurrent palpitations past 3 mo la...
2        dillon cleveland 17 y.o. male patient signific...
3        17 yo c/o palpitation started 3 mos ago; . not...
4        17yo male pmh evaluation palpitations. states ...
                               ...                        
42141    ms. madden 20 yo female presenting worst ha li...
42142    20 yo f came complain dull 8/10 headache assoc...
42143    ms. madden 20yo female presents headache 1 day...
42144    stephanie madden 20 year old woman complaining...
42145    patient 20 yo f presents headache. said never ...
Name: text, Length: 42146, dtype: object

In [10]:
data = pd.merge(patient_notes, labels)
data

,pn_num,case_num,pn_history,text,feature_num
0,16,0,HPI: 17yo M presents with palpitations. Patien...,hpi: 17yo presents palpitations. patient repor...,"[0, 1, 2, 3, 4, 6, 9, 10, 11, 12]"
1,41,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,17 y/o came clinic c/o heart pounding. started...,"[1, 2, 3, 7, 9, 10, 11, 12]"
2,46,0,Mr. Cleveland is a 17yo M who was consented by...,mr. cleveland 17yo consented parents alone tod...,"[0, 1, 3, 4, 5, 6, 7, 9, 10, 11, 12]"
3,82,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,17 yo cardiac arrhythmia pmh presents complain...,"[0, 1, 3, 4, 6, 7, 9, 10, 11, 12]"
4,100,0,HPI: Dillon Cleveland is an otherwise healthy ...,hpi: dillon cleveland otherwise healthy 17 yea...,"[0, 1, 2, 3, 4, 6, 9, 10, 11, 12]"
...,...,...,...,...,...
995,95145,9,Pt is 20 yo F w headache since yesterday morni...,pt 20 yo f w headache since yesterday morning....,"[901, 902, 904, 905, 906, 908, 912, 913, 914, ..."
996,95228,9,"20 F no PMH, lives w/ roommate in apartment ha...","20 f pmh, lives roommate apartment severe head...","[901, 902, 909, 910, 913, 914, 915]"
997,95243,9,20 y/o F with no PMH is presenting with 1 day ...,20 y/o f pmh presenting 1 day history headache...,"[900, 901, 902, 906, 908, 912, 913, 914, 916]"
998,95330,9,Ms. Madden is a 20 yo female presenting w/ the...,ms. madden 20 yo female presenting worst ha li...,"[900, 901, 902, 904, 906, 907, 908, 912, 913, ..."


In [11]:
data = data.drop(columns=["pn_history"])

In [12]:
data

,pn_num,case_num,text,feature_num
0,16,0,hpi: 17yo presents palpitations. patient repor...,"[0, 1, 2, 3, 4, 6, 9, 10, 11, 12]"
1,41,0,17 y/o came clinic c/o heart pounding. started...,"[1, 2, 3, 7, 9, 10, 11, 12]"
2,46,0,mr. cleveland 17yo consented parents alone tod...,"[0, 1, 3, 4, 5, 6, 7, 9, 10, 11, 12]"
3,82,0,17 yo cardiac arrhythmia pmh presents complain...,"[0, 1, 3, 4, 6, 7, 9, 10, 11, 12]"
4,100,0,hpi: dillon cleveland otherwise healthy 17 yea...,"[0, 1, 2, 3, 4, 6, 9, 10, 11, 12]"
...,...,...,...,...
995,95145,9,pt 20 yo f w headache since yesterday morning....,"[901, 902, 904, 905, 906, 908, 912, 913, 914, ..."
996,95228,9,"20 f pmh, lives roommate apartment severe head...","[901, 902, 909, 910, 913, 914, 915]"
997,95243,9,20 y/o f pmh presenting 1 day history headache...,"[900, 901, 902, 906, 908, 912, 913, 914, 916]"
998,95330,9,ms. madden 20 yo female presenting worst ha li...,"[900, 901, 902, 904, 906, 907, 908, 912, 913, ..."


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [14]:




tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords)


In [15]:
tfidf_matrix = tfidf_vectorizer.fit_transform(data['text'])
tfidf_matrix

<1000x5717 sparse matrix of type '<class 'numpy.float64'>'
	with 77215 stored elements in Compressed Sparse Row format>

In [16]:
keywords = tfidf_vectorizer.get_feature_names_out()


In [17]:
keyword_lists = []

for i, row in data.iterrows():
    tfidf_scores = zip(keywords, tfidf_matrix[i].toarray()[0])
    keywords_list = [keyword for keyword, _ in sorted(tfidf_scores, key=lambda x: x[1], reverse=True) if _ > 0][:10]
    keyword_lists.append(keywords_list)

In [18]:
keyword_df = pd.DataFrame({'keywords': keyword_lists})
keyword_df

,keywords
0,"[game, adderrall, soccer, endorses, abusing, a..."
1,"[experiencing, goes, mo, aterol, sharing, star..."
2,"[basketball, adderall, today, palpitations, co..."
3,"[aderall, accept, prescribed, pt, adverse, arr..."
4,"[pounding, beat, dose, tachycardia, rapid, unk..."
...,...
995,"[headache, getthing, high, auras, cannabis, co..."
996,"[x2, instrumentation, spinal, photosensitivity..."
997,"[patient, soical, sensitive, said, headache, v..."
998,"[ha, felt, hpl, morn, nonpulsating, unlike, ye..."


In [19]:
result_df = pd.concat([data, keyword_df], axis=1)
result_df

,pn_num,case_num,text,feature_num,keywords
0,16,0,hpi: 17yo presents palpitations. patient repor...,"[0, 1, 2, 3, 4, 6, 9, 10, 11, 12]","[game, adderrall, soccer, endorses, abusing, a..."
1,41,0,17 y/o came clinic c/o heart pounding. started...,"[1, 2, 3, 7, 9, 10, 11, 12]","[experiencing, goes, mo, aterol, sharing, star..."
2,46,0,mr. cleveland 17yo consented parents alone tod...,"[0, 1, 3, 4, 5, 6, 7, 9, 10, 11, 12]","[basketball, adderall, today, palpitations, co..."
3,82,0,17 yo cardiac arrhythmia pmh presents complain...,"[0, 1, 3, 4, 6, 7, 9, 10, 11, 12]","[aderall, accept, prescribed, pt, adverse, arr..."
4,100,0,hpi: dillon cleveland otherwise healthy 17 yea...,"[0, 1, 2, 3, 4, 6, 9, 10, 11, 12]","[pounding, beat, dose, tachycardia, rapid, unk..."
...,...,...,...,...,...
995,95145,9,pt 20 yo f w headache since yesterday morning....,"[901, 902, 904, 905, 906, 908, 912, 913, 914, ...","[headache, getthing, high, auras, cannabis, co..."
996,95228,9,"20 f pmh, lives roommate apartment severe head...","[901, 902, 909, 910, 913, 914, 915]","[x2, instrumentation, spinal, photosensitivity..."
997,95243,9,20 y/o f pmh presenting 1 day history headache...,"[900, 901, 902, 906, 908, 912, 913, 914, 916]","[patient, soical, sensitive, said, headache, v..."
998,95330,9,ms. madden 20 yo female presenting worst ha li...,"[900, 901, 902, 904, 906, 907, 908, 912, 913, ...","[ha, felt, hpl, morn, nonpulsating, unlike, ye..."


In [20]:
features["feature_text"] = features["feature_text"].str.lower()

In [45]:
features['feature_text'] = features['feature_text'].str.replace('-', ' ', regex=False)
features


,feature_num,case_num,feature_text,num
0,0,0,family history of mi or family history of myoc...,0
1,1,0,family history of thyroid disorder,1
2,2,0,chest pressure,2
3,3,0,intermittent symptoms,3
4,4,0,lightheaded,4
...,...,...,...,...
137,911,9,meningococcal vaccine status unknown,126
138,912,9,family history of migraines,127
140,914,9,photophobia,128
141,915,9,no known illness contacts,129


In [22]:
features = features.drop_duplicates(subset="feature_text")

In [23]:
features

,feature_num,case_num,feature_text
0,0,0,family-history-of-mi-or-family-history-of-myoc...
1,1,0,family-history-of-thyroid-disorder
2,2,0,chest-pressure
3,3,0,intermittent-symptoms
4,4,0,lightheaded
...,...,...,...
137,911,9,meningococcal-vaccine-status-unknown
138,912,9,family-history-of-migraines
140,914,9,photophobia
141,915,9,no-known-illness-contacts


In [24]:
import numpy as np


def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

In [93]:
glove_file = 'glove-wikipedia/glove.6B.300d.txt'  # Path to the GloVe file
embeddings_index = load_glove_embeddings(glove_file)

In [94]:
def vectorize_word(word, embeddings_index):
    return embeddings_index.get(word, np.zeros((300,)))

In [95]:
def vectorize_phrase(phrase, embeddings_index):
    words = phrase.split()
    vectors = [vectorize_word(word, embeddings_index) for word in words]
    return np.mean(vectors, axis=0)

In [96]:
from sklearn.metrics.pairwise import cosine_similarity


In [97]:
first_data = result_df["keywords"][0]

In [98]:
labels_vector_array = []
for feature in features["feature_text"]:
    labels_vector_array.append((vectorize_phrase(feature, embeddings_index)))

In [99]:
def make_vector_array(row):
    keyword_vectors = []
    for w in row:
        keyword_vectors.append((vectorize_word(w, embeddings_index)))
    
    similarity_matrix = cosine_similarity(keyword_vectors, labels_vector_array)
    highest_similarity_indices = np.argmax(similarity_matrix, axis=1)
    return highest_similarity_indices

In [100]:
results = []
for row in result_df["keywords"]:
    results.append(make_vector_array(row))
results

[array([ 76,   0,  69,   6,  29,   0, 128,  12,  14,  59], dtype=int64),
 array([  3,  85,  71,   0, 116,  90,  18,   1,  27,  88], dtype=int64),
 array([69,  6, 48, 55, 75, 96, 70, 30, 49, 43], dtype=int64),
 array([  0,  54, 109,  60,  32,  63, 125,   0,   6,  39], dtype=int64),
 array([ 39,  49,   8,  63,  81, 126,  22, 111,  76,   9], dtype=int64),
 array([  0,  31,   0,  31,  31,   0,   0,  86,  55, 128], dtype=int64),
 array([  0,   0, 129,   9, 109,   0,   0,   0,  34,  39], dtype=int64),
 array([ 55,  76,   0, 104,  87, 105,  94,   4,  10,  92], dtype=int64),
 array([55, 92,  0,  0, 28, 34, 72, 16, 75, 55], dtype=int64),
 array([ 70,   8,   9,  55, 103,  54,  28, 121,  28, 105], dtype=int64),
 array([  0,   0, 121,  44,  48,   9, 103,   5,  87,   4], dtype=int64),
 array([  0,   0,  31, 129,   0,  28,   0, 103,  43,  75], dtype=int64),
 array([ 6,  0,  0,  0,  0, 28, 82,  0, 55, 70], dtype=int64),
 array([  8,   0,  69,  77, 101,   6,  58,  24, 109,  59], dtype=int64),
 array([

In [101]:
predicted = []
for result in results:
    row_result = []
    for number in result:
        row_result.append(features.iloc[number]["feature_num"])
    predicted.append(row_result)
predicted

[[511, 0, 504, 6, 203, 0, 914, 12, 101, 402],
 [3, 604, 506, 0, 817, 609, 105, 1, 201, 607],
 [504, 6, 306, 314, 510, 704, 505, 204, 307, 301],
 [0, 313, 810, 403, 206, 406, 910, 0, 6, 214],
 [214, 307, 8, 406, 516, 911, 109, 812, 511, 9],
 [0, 205, 0, 205, 205, 0, 0, 605, 314, 914],
 [0, 0, 915, 9, 810, 0, 0, 0, 209, 214],
 [314, 511, 0, 805, 606, 806, 702, 4, 10, 611],
 [314, 611, 0, 0, 202, 209, 507, 103, 510, 314],
 [505, 8, 9, 314, 804, 313, 202, 905, 202, 806],
 [0, 0, 905, 302, 306, 9, 804, 5, 606, 4],
 [0, 0, 205, 915, 0, 202, 0, 804, 301, 510],
 [6, 0, 0, 0, 0, 202, 517, 0, 314, 505],
 [8, 0, 504, 512, 801, 6, 401, 111, 810, 402],
 [706, 6, 0, 0, 0, 912, 0, 315, 0, 505],
 [0, 11, 505, 504, 801, 804, 402, 0, 101, 504],
 [0, 402, 505, 0, 600, 511, 307, 505, 505, 1],
 [914, 706, 207, 0, 0, 907, 810, 9, 502, 810],
 [5, 0, 0, 804, 804, 910, 105, 810, 311, 112],
 [312, 0, 514, 313, 306, 306, 502, 0, 914, 212],
 [2, 607, 103, 0, 214, 609, 609, 8, 504, 609],
 [5, 504, 507, 313, 307, 8

In [102]:
labels

,case_num,pn_num,feature_num
0,0,16,"[0, 1, 2, 3, 4, 6, 9, 10, 11, 12]"
1,0,41,"[1, 2, 3, 7, 9, 10, 11, 12]"
2,0,46,"[0, 1, 3, 4, 5, 6, 7, 9, 10, 11, 12]"
3,0,82,"[0, 1, 3, 4, 6, 7, 9, 10, 11, 12]"
4,0,100,"[0, 1, 2, 3, 4, 6, 9, 10, 11, 12]"
...,...,...,...
995,9,95145,"[901, 902, 904, 905, 906, 908, 912, 913, 914, ..."
996,9,95228,"[901, 902, 909, 910, 913, 914, 915]"
997,9,95243,"[900, 901, 902, 906, 908, 912, 913, 914, 916]"
998,9,95330,"[900, 901, 902, 904, 906, 907, 908, 912, 913, ..."


In [103]:
features["num"] = range(len(features))
features

,feature_num,case_num,feature_text,num
0,0,0,family history of mi or family history of myoc...,0
1,1,0,family history of thyroid disorder,1
2,2,0,chest pressure,2
3,3,0,intermittent symptoms,3
4,4,0,lightheaded,4
...,...,...,...,...
137,911,9,meningococcal vaccine status unknown,126
138,912,9,family history of migraines,127
140,914,9,photophobia,128
141,915,9,no known illness contacts,129


In [104]:
index = features[features['feature_num'] == 112]["num"].iloc[0]
index

25

In [105]:
true_indices = []
for row in labels["feature_num"]:
    index_row = []
    for number in row:
        try:
            index_row.append(features[features['feature_num'] == number]["num"].iloc[0])
        except IndexError:
            print(f"IndexError: No matching 'num' found for 'feature_num' {number}")
          # Add a placeholder or handle the error as appropriate
    true_indices.append(index_row)


IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No matching 'num' found for 'feature_num' 208
IndexError: No

In [106]:
true_indices

[[0, 1, 2, 3, 4, 6, 9, 10, 11, 12],
 [1, 2, 3, 7, 9, 10, 11, 12],
 [0, 1, 3, 4, 5, 6, 7, 9, 10, 11, 12],
 [0, 1, 3, 4, 6, 7, 9, 10, 11, 12],
 [0, 1, 2, 3, 4, 6, 9, 10, 11, 12],
 [0, 1, 2, 3, 7, 9, 10, 11, 12],
 [0, 1, 2, 3, 6, 9, 10, 11, 12],
 [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12],
 [0, 1, 2, 3, 4, 6, 9, 11, 12],
 [0, 1, 3, 5, 6, 7, 8, 9, 10, 11, 12],
 [0, 1, 3, 4, 6, 9, 10, 11, 12],
 [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12],
 [0, 1, 2, 6, 7, 9, 10, 12],
 [0, 1, 2, 6, 9, 10, 11, 12],
 [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12],
 [0, 1, 2, 3, 6, 9, 10, 11, 12],
 [0, 1, 2, 3, 6, 7, 9, 10, 11, 12],
 [0, 1, 3, 4, 6, 7, 9, 10, 11, 12],
 [0, 1, 2, 3, 4, 6, 7, 9, 10, 11, 12],
 [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12],
 [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12],
 [0, 2, 3, 4, 6, 8, 9, 10, 11, 12],
 [0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12],
 [0, 1, 2, 3, 4, 7, 9, 11, 12],
 [0, 1, 2, 7, 9, 11, 12],
 [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12],
 [0, 1, 3, 5, 6, 8, 9, 10, 11, 12],
 [0, 1, 3, 5, 6, 9, 10, 11, 12],
 [0, 1, 

In [107]:
length = 131

predicted_ary = []
for row in results:
    binary_labels = [0] * length
    for index in row:
        binary_labels[index] = 1
    predicted_ary.append(binary_labels)


In [108]:
length = 131

true_ary = []
for row in true_indices:
    binary_labels = [0] * length
    for index in row:
        binary_labels[index] = 1
    true_ary.append(binary_labels)

In [109]:


from sklearn.metrics import f1_score, precision_score, recall_score


macro_precision = precision_score(true_ary, predicted_ary, average='macro', zero_division=0)
macro_recall = recall_score(true_ary, predicted_ary, average='macro', zero_division=0)
macro_f1 = f1_score(true_ary, predicted_ary, average='macro', zero_division=0)

In [110]:
macro_f1, macro_recall, macro_precision

(0.11727027359763811, 0.1466758212560681, 0.15559690950969632)

In [111]:
macro_precision = precision_score(true_ary, predicted_ary, average='micro', zero_division=0)
macro_recall = recall_score(true_ary, predicted_ary, average='micro', zero_division=0)
macro_f1 = f1_score(true_ary, predicted_ary, average='micro', zero_division=0)

In [112]:
macro_f1, macro_recall, macro_precision

(0.13923444976076554, 0.13289188263500398, 0.1462127873382741)